In [1]:
import requests 
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd


In [47]:
# Create a python funtion to get all product links from walmart
headers = { 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}

result = []

def walmart(name):
    Walmart = 'https://www.walmart.com/search?q='
    page = Request(Walmart + name, headers=headers)
    html = urlopen(page).read()
    soup = BeautifulSoup(html, 'html.parser')

    results = soup.find_all('a', class_="absolute w-100 h-100 z-1" )

    try: 
        for link in results:
            if 'href' in link.attrs and 'https' in link.attrs['href']:
                result.append(str(link.attrs['href']))
                
            else:
                result.append('https://www.walmart.com/' + str(link.attrs['href']))

                
                        
    except AttributeError:
            print("title: -")


#CREATE a function to scrap the product name, price and product image link from Walmart
final_list = []

def get_items_price(urls):

    try:
        for url in urls:
            pages = Request(url, headers=headers)
            html = urlopen(pages).read()
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find(class_="f3 b lh-copy dark-gray mt1 mb2").get_text()
            price = soup.find(itemprop="price").get_text()

            for item in soup.find_all('img', loading="lazy"):
                    
                images = item['src']
            items = title, price, images

            final_list.append(items)
                        
    except AttributeError:
        print("title: -")
        
    return 

walmart('ProductName')

get_items_price(result)



In [62]:
#Create a dataframe for the product name, price and the product image link 
import urllib

df=pd.DataFrame(final_list, columns =['Item name', 'Price', 'Item image link'])

#add the product link to the dataframe
df['Item link'] = result


In [61]:
# Create a python funtion to get all product links from Amazon

headers = { 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"}

result = []

def amazon(name):
    Walmart = 'https://www.amazon.com/s?k='
    page = Request(Walmart + name, headers=headers)
    html = urlopen(page).read()
    soup = BeautifulSoup(html, 'html.parser')

    results = soup.find_all('a', class_="a-link-normal s-no-outline" )

    try: 
        for link in results:
            if 'href' in link.attrs and 'https' in link.attrs['href']:
                result.append(str(link.attrs['href']))
                
            else:
                result.append('https://www.amazon.com/' + str(link.attrs['href']))

                
                        
    except AttributeError:
            print("title: -")


amazon('ProductName')

#CREATE a function to scrap the product name, price and product image link from Amazon

final_list = []

def get_items_price(urls):

    try:
        for url in urls:
            pages = Request(url, headers=headers)
            html = urlopen(pages).read()
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find(id="productTitle").get_text().strip()
#             print(title)
            price = soup.find(id="priceblock_ourprice").get_text()
#             print(price)

            for item in soup.find_all('img', attrs={"id":'landingImage'}):                    
                images = item['src']
#                 print(images)
            
#             print(title, price, images)
            items = title, price, images

            final_list.append(items)
                        
    except AttributeError:
        print("title: -")
        
    return 

get_items_price(result)

# print(final_list)

title: -


In [63]:
import urllib

amz=pd.DataFrame(final_list, columns =['Item name', 'Price', 'Item image link'])

amz['Item link'] = pd.Series(result)
xlsx_file = amz.to_excel('ProductName.xlsx', index = True)



In [50]:
#Put the two dataframes from Walmart and Amazon together
pieces= {'Walmart': df, 'Amazon': amz}

result = pd.concat(pieces)

#Export the data frame into an excel file
xlsx_file = result.to_excel('ProductName.xlsx', index = True)


In [60]:
#Use python function to send the excel file to an email address
import smtplib
from email.message import EmailMessage

SENDER_EMAIL = "youremail@gmail.com"
APP_PASSWORD = "yourPassword"

def send_mail_with_excel(recipient_email, subject, content, excel_file):
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = recipient_email
    msg.set_content(content)

    with open(excel_file, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename=excel_file)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(SENDER_EMAIL, APP_PASSWORD)
        smtp.send_message(msg)
        
recipient = 'yourRecipientemail@gmail.com'
subject = 'subject'
content = 'content'
send_mail_with_excel(recipient, subject, content, 'ProductName.xlsx' )